In [1]:
import pandas as pd 
import yfinance as yf
import numpy as np
import random
from scipy.optimize import minimize
from sklearn.linear_model import LinearRegression

In [2]:
# custom data set
dataset = pd.read_csv('dataset.csv')
dataset['stock'].unique()

array(['AAPL', 'ABBV', 'AMZN', 'DB', 'DIS', 'FB', 'GOOG', 'HAL', 'HSBC',
       'JPM', 'KO', 'MCD', 'MSFT', 'PFE', 'XOM'], dtype=object)

In [3]:
dataset

,Date,stock,adj_close,bbr,pnlog
0,2015-01-02,AAPL,99.945885,58.343195,0.010336
1,2015-01-05,AAPL,97.130241,65.384615,0.014229
2,2015-01-06,AAPL,97.139420,73.206751,0.017950
3,2015-01-07,AAPL,98.501518,67.732123,0.000000
4,2015-01-08,AAPL,102.286186,62.257496,0.005537
...,...,...,...,...,...
20515,2020-06-03,XOM,49.240002,NaN,NaN
20516,2020-06-04,XOM,49.099998,37.168142,-0.011122
20517,2020-06-05,XOM,53.080002,39.331027,-0.017159
20518,2020-06-08,XOM,54.740002,100.000000,0.000000


In [4]:
# pull the data ourselfs
def get_data(tickers, start_date, end_date):

    # initialise list to store ticker data frames
    dfs = []

    for ticker in tickers:
        # Download the stock price data with yfinance
        data = yf.download(ticker, start=start_date, end=end_date, interval='1d')
    
        # Create a new data frame with the necessary columns
        df = pd.DataFrame(index=data.index)
        df["ticker"] = ticker
        df["adj_close"] = data["Adj Close"]
        df["adj_close_lag"] = data["Adj Close"].shift(1)
        df["return"] = ((df["adj_close"] / df["adj_close_lag"]) - 1)*100
        df = df[["ticker", "return"]]
        dfs.append(df)

    # Concatenate the data frames vertically
    result = pd.concat(dfs)
    result = result.dropna()

    return result

In [5]:
ticker = ['AAPL', 'ABBV', 'AMZN', 'DB', 'DIS', 'FB', 'GOOG', 'HAL', 'HSBC', 'JPM', 'KO', 'MCD', 'MSFT', 'PFE', 'XOM']

stocks = get_data(ticker, '2015-01-01', '2020-06-01')
stocks

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- FB: No timezone found, symbol may be delisted
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%*********

,ticker,return
Date,,
2015-01-05,AAPL,-2.817130
2015-01-06,AAPL,0.009416
2015-01-07,AAPL,1.402210
2015-01-08,AAPL,3.842247
2015-01-09,AAPL,0.107233
...,...,...
2020-05-22,XOM,0.089767
2020-05-26,XOM,2.937226
2020-05-27,XOM,0.718785


In [6]:
stocks.reset_index(inplace=True)

# Pivot the data frame to invert it
inverted_df = stocks.pivot(index='Date', columns='ticker', values='return')

# Reset the column index
inverted_df.columns.name = None

inverted_df = inverted_df.reset_index()
inverted_df

,Date,AAPL,ABBV,AMZN,DB,DIS,GOOG,HAL,HSBC,JPM,KO,MCD,MSFT,PFE,XOM
0,2015-01-05,-2.817130,-1.881942,-2.051729,-5.415164,-1.461356,-2.084562,-1.975207,-2.294430,-3.104486,0.000000,-1.104470,-0.919569,-0.542577,-2.736237
1,2015-01-06,0.009416,-0.494983,-2.283333,-1.596118,-0.530411,-2.317709,-0.981684,-1.456844,-2.592927,0.759365,0.184339,-1.467725,0.834400,-0.531599
2,2015-01-07,1.402210,4.041668,1.059974,1.057833,1.022978,-0.171323,2.687214,0.772271,0.152573,1.248225,1.742432,1.270507,1.368553,1.013224
3,2015-01-08,3.842247,1.045858,0.683602,0.209339,1.034136,0.315304,2.159561,0.328459,2.234649,1.209552,0.372315,2.941830,2.040809,1.664544
4,2015-01-09,0.107233,-2.735468,-1.174861,-1.671300,0.490457,-1.295055,-1.019654,-0.523771,-1.738685,-1.103178,-1.218759,-0.840491,0.461530,-0.141012
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1355,2020-05-22,0.643836,-0.989031,-0.402982,-0.543476,0.161245,0.543205,-3.255411,-5.860658,-0.776297,-0.309955,-0.361995,0.043611,0.644120,0.089767
1356,2020-05-26,-0.677332,-1.509250,-0.616361,12.158465,2.482630,0.467937,2.415865,3.656941,7.097345,2.353986,0.233167,-1.057146,-0.026647,2.937226
1357,2020-05-27,0.435693,-0.804750,-0.473604,5.724739,0.479539,0.057868,2.695875,1.511959,5.792112,1.388598,1.558102,0.132166,-0.213403,0.718785
1358,2020-05-28,0.043990,0.055573,-0.385415,-1.958531,-3.933184,-0.078283,-0.656276,-2.399655,-1.489591,0.770364,0.538034,-0.225520,2.058258,-2.595153


In [7]:
dataset_tr = dataset.pivot(index='Date', columns='stock', values='pnlog')

# Reset the column index
dataset_tr.columns.name = None

dataset_tr = dataset_tr.reset_index()
dataset_tr

,Date,AAPL,ABBV,AMZN,DB,DIS,FB,GOOG,HAL,HSBC,JPM,KO,MCD,MSFT,PFE,XOM
0,2015-01-02,0.010336,-0.015882,-0.020067,0.004023,0.008930,-0.010094,0.007672,0.001855,-0.037944,0.022862,NaN,-0.001835,0.011189,-0.015655,NaN
1,2015-01-05,0.014229,0.000000,0.013374,0.008442,0.010258,0.011148,-0.002559,-0.012387,-0.023388,0.007260,0.000000,-0.001565,0.013213,0.000000,-0.023893
2,2015-01-06,0.017950,0.010092,0.019457,-0.005736,-0.016278,0.007027,0.017585,0.019216,-0.035777,-0.023960,0.000000,-0.037644,0.017114,0.010571,-0.026224
3,2015-01-07,0.000000,0.010801,-0.010497,0.010894,-0.002005,0.010447,-0.004112,0.003567,-0.021697,-0.013968,0.000000,-0.017531,0.020706,0.012930,0.000000
4,2015-01-08,0.005537,0.007774,0.010360,-0.003153,0.041047,-0.003588,0.015942,0.001991,-0.009976,0.001864,0.007091,-0.000143,0.028966,0.008093,-0.003405
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1363,2020-06-03,0.000284,-0.027009,-0.017542,0.000000,-0.039138,-0.060086,-0.030677,-0.016068,-0.017556,0.006868,0.000000,0.000000,0.000000,-0.002122,NaN
1364,2020-06-04,-0.004683,-0.033180,-0.014703,0.000000,-0.007554,-0.034237,-0.023413,-0.016782,-0.012212,-0.006033,NaN,0.000000,-0.020826,-0.016932,-0.011122
1365,2020-06-05,-0.009095,0.059226,-0.031269,-0.013804,-0.007840,0.000000,-0.011109,-0.009981,-0.015163,-0.004168,0.004322,0.004753,-0.016721,0.000997,-0.017159
1366,2020-06-08,0.001563,NaN,0.007215,0.035353,0.000000,-0.003304,0.001961,-0.017587,-0.024399,-0.007609,0.000000,0.000000,-0.004798,0.009710,0.000000


In [8]:
dataset_tr = dataset_tr.drop(columns=['FB'], axis=1)

new_c = []
for d in dataset_tr:
    new_c.append(f'{d}_pnlog')

dataset_tr.columns = new_c

dataset_tr

,Date_pnlog,AAPL_pnlog,ABBV_pnlog,AMZN_pnlog,DB_pnlog,DIS_pnlog,GOOG_pnlog,HAL_pnlog,HSBC_pnlog,JPM_pnlog,KO_pnlog,MCD_pnlog,MSFT_pnlog,PFE_pnlog,XOM_pnlog
0,2015-01-02,0.010336,-0.015882,-0.020067,0.004023,0.008930,0.007672,0.001855,-0.037944,0.022862,NaN,-0.001835,0.011189,-0.015655,NaN
1,2015-01-05,0.014229,0.000000,0.013374,0.008442,0.010258,-0.002559,-0.012387,-0.023388,0.007260,0.000000,-0.001565,0.013213,0.000000,-0.023893
2,2015-01-06,0.017950,0.010092,0.019457,-0.005736,-0.016278,0.017585,0.019216,-0.035777,-0.023960,0.000000,-0.037644,0.017114,0.010571,-0.026224
3,2015-01-07,0.000000,0.010801,-0.010497,0.010894,-0.002005,-0.004112,0.003567,-0.021697,-0.013968,0.000000,-0.017531,0.020706,0.012930,0.000000
4,2015-01-08,0.005537,0.007774,0.010360,-0.003153,0.041047,0.015942,0.001991,-0.009976,0.001864,0.007091,-0.000143,0.028966,0.008093,-0.003405
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1363,2020-06-03,0.000284,-0.027009,-0.017542,0.000000,-0.039138,-0.030677,-0.016068,-0.017556,0.006868,0.000000,0.000000,0.000000,-0.002122,NaN
1364,2020-06-04,-0.004683,-0.033180,-0.014703,0.000000,-0.007554,-0.023413,-0.016782,-0.012212,-0.006033,NaN,0.000000,-0.020826,-0.016932,-0.011122
1365,2020-06-05,-0.009095,0.059226,-0.031269,-0.013804,-0.007840,-0.011109,-0.009981,-0.015163,-0.004168,0.004322,0.004753,-0.016721,0.000997,-0.017159
1366,2020-06-08,0.001563,NaN,0.007215,0.035353,0.000000,0.001961,-0.017587,-0.024399,-0.007609,0.000000,0.000000,-0.004798,0.009710,0.000000


In [9]:
interval = '1d'
start= '2015-01-01'
end= '2020-06-09'

In [10]:
sp500 = yf.download('SPY', start=start, end=end, interval='1d')
sp500

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2015-01-02,206.380005,206.880005,204.179993,205.429993,176.134598,121465900
2015-01-05,204.169998,204.369995,201.350006,201.720001,172.953674,169632600
2015-01-06,202.089996,202.720001,198.860001,199.820007,171.324615,209151400
2015-01-07,201.419998,202.720001,200.880005,202.309998,173.459549,125346700
2015-01-08,204.009995,206.160004,203.990005,205.899994,176.537613,147217800
...,...,...,...,...,...,...
2020-06-02,306.549988,308.130005,305.100006,308.079987,293.203857,74267200
2020-06-03,310.239990,313.220001,309.940002,312.179993,297.105865,92567600
2020-06-04,311.109985,313.000000,309.079987,311.359985,296.325470,75794400


In [11]:
sp500 = sp500.rename(columns={"Adj Close": "adjclose"})
sp500 = sp500[['adjclose']]
sp500

,adjclose
Date,
2015-01-02,176.134598
2015-01-05,172.953674
2015-01-06,171.324615
2015-01-07,173.459549
2015-01-08,176.537613
...,...
2020-06-02,293.203857
2020-06-03,297.105865
2020-06-04,296.325470


In [12]:
sp500['SP500_return'] = (sp500['adjclose'] / sp500['adjclose'].shift(1) - 1) * 100

In [13]:
# Convert 'date' column in df2 to datetime (if it's not already)
dataset['Date'] = pd.to_datetime(dataset['Date'])

# Set 'date' column as the index in df2
dataset.set_index('Date', inplace=True)

# Join the two dataframes
df = sp500.join(dataset, how='inner')  # Use how='left' for left join, 'outer' for outer join, etc.

In [14]:
# Read the file into a DataFrame, skipping the last row
ff = pd.read_csv('F-F_Research_Data_Factors_daily.CSV', skiprows= 4, skipfooter=1, engine='python', index_col=0)

In [15]:
ff['Date'] = ff.index
ff['Date'] = pd.to_datetime(ff.index, format='%Y%m%d')
ff = ff.set_index('Date')

In [16]:
ff

,Mkt-RF,SMB,HML,RF
Date,,,,
1926-07-01,0.10,-0.24,-0.28,0.009
1926-07-02,0.45,-0.32,-0.08,0.009
1926-07-06,0.17,0.27,-0.35,0.009
1926-07-07,0.09,-0.59,0.03,0.009
1926-07-08,0.21,-0.36,0.15,0.009
...,...,...,...,...
2021-01-25,0.28,-0.07,-0.45,0.000
2021-01-26,-0.32,0.00,0.03,0.000
2021-01-27,-2.53,1.30,1.47,0.000


In [17]:
df = pd.merge(ff, inverted_df, on='Date', how='right')
df

,Date,Mkt-RF,SMB,HML,RF,AAPL,ABBV,AMZN,DB,DIS,GOOG,HAL,HSBC,JPM,KO,MCD,MSFT,PFE,XOM
0,2015-01-05,-1.84,0.34,-0.68,0.0,-2.817130,-1.881942,-2.051729,-5.415164,-1.461356,-2.084562,-1.975207,-2.294430,-3.104486,0.000000,-1.104470,-0.919569,-0.542577,-2.736237
1,2015-01-06,-1.04,-0.78,-0.30,0.0,0.009416,-0.494983,-2.283333,-1.596118,-0.530411,-2.317709,-0.981684,-1.456844,-2.592927,0.759365,0.184339,-1.467725,0.834400,-0.531599
2,2015-01-07,1.19,0.18,-0.64,0.0,1.402210,4.041668,1.059974,1.057833,1.022978,-0.171323,2.687214,0.772271,0.152573,1.248225,1.742432,1.270507,1.368553,1.013224
3,2015-01-08,1.81,-0.12,-0.28,0.0,3.842247,1.045858,0.683602,0.209339,1.034136,0.315304,2.159561,0.328459,2.234649,1.209552,0.372315,2.941830,2.040809,1.664544
4,2015-01-09,-0.85,0.01,-0.48,0.0,0.107233,-2.735468,-1.174861,-1.671300,0.490457,-1.295055,-1.019654,-0.523771,-1.738685,-1.103178,-1.218759,-0.840491,0.461530,-0.141012
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1355,2020-05-22,0.27,0.47,-0.87,0.0,0.643836,-0.989031,-0.402982,-0.543476,0.161245,0.543205,-3.255411,-5.860658,-0.776297,-0.309955,-0.361995,0.043611,0.644120,0.089767
1356,2020-05-26,1.23,0.04,4.59,0.0,-0.677332,-1.509250,-0.616361,12.158465,2.482630,0.467937,2.415865,3.656941,7.097345,2.353986,0.233167,-1.057146,-0.026647,2.937226
1357,2020-05-27,1.54,0.61,3.62,0.0,0.435693,-0.804750,-0.473604,5.724739,0.479539,0.057868,2.695875,1.511959,5.792112,1.388598,1.558102,0.132166,-0.213403,0.718785
1358,2020-05-28,-0.41,-1.47,-2.41,0.0,0.043990,0.055573,-0.385415,-1.958531,-3.933184,-0.078283,-0.656276,-2.399655,-1.489591,0.770364,0.538034,-0.225520,2.058258,-2.595153


In [18]:
ticker = ['AAPL', 'ABBV', 'AMZN', 'DB', 'DIS', 'GOOG', 'HAL', 'HSBC', 'JPM', 'KO', 'MCD', 'MSFT', 'PFE', 'XOM']

In [19]:
for t in ticker:
    df[f'{t}_target'] = df[t]-df['RF']

In [20]:
df

,Date,Mkt-RF,SMB,HML,RF,AAPL,ABBV,AMZN,DB,DIS,...,DIS_target,GOOG_target,HAL_target,HSBC_target,JPM_target,KO_target,MCD_target,MSFT_target,PFE_target,XOM_target
0,2015-01-05,-1.84,0.34,-0.68,0.0,-2.817130,-1.881942,-2.051729,-5.415164,-1.461356,...,-1.461356,-2.084562,-1.975207,-2.294430,-3.104486,0.000000,-1.104470,-0.919569,-0.542577,-2.736237
1,2015-01-06,-1.04,-0.78,-0.30,0.0,0.009416,-0.494983,-2.283333,-1.596118,-0.530411,...,-0.530411,-2.317709,-0.981684,-1.456844,-2.592927,0.759365,0.184339,-1.467725,0.834400,-0.531599
2,2015-01-07,1.19,0.18,-0.64,0.0,1.402210,4.041668,1.059974,1.057833,1.022978,...,1.022978,-0.171323,2.687214,0.772271,0.152573,1.248225,1.742432,1.270507,1.368553,1.013224
3,2015-01-08,1.81,-0.12,-0.28,0.0,3.842247,1.045858,0.683602,0.209339,1.034136,...,1.034136,0.315304,2.159561,0.328459,2.234649,1.209552,0.372315,2.941830,2.040809,1.664544
4,2015-01-09,-0.85,0.01,-0.48,0.0,0.107233,-2.735468,-1.174861,-1.671300,0.490457,...,0.490457,-1.295055,-1.019654,-0.523771,-1.738685,-1.103178,-1.218759,-0.840491,0.461530,-0.141012
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1355,2020-05-22,0.27,0.47,-0.87,0.0,0.643836,-0.989031,-0.402982,-0.543476,0.161245,...,0.161245,0.543205,-3.255411,-5.860658,-0.776297,-0.309955,-0.361995,0.043611,0.644120,0.089767
1356,2020-05-26,1.23,0.04,4.59,0.0,-0.677332,-1.509250,-0.616361,12.158465,2.482630,...,2.482630,0.467937,2.415865,3.656941,7.097345,2.353986,0.233167,-1.057146,-0.026647,2.937226
1357,2020-05-27,1.54,0.61,3.62,0.0,0.435693,-0.804750,-0.473604,5.724739,0.479539,...,0.479539,0.057868,2.695875,1.511959,5.792112,1.388598,1.558102,0.132166,-0.213403,0.718785
1358,2020-05-28,-0.41,-1.47,-2.41,0.0,0.043990,0.055573,-0.385415,-1.958531,-3.933184,...,-3.933184,-0.078283,-0.656276,-2.399655,-1.489591,0.770364,0.538034,-0.225520,2.058258,-2.595153


In [21]:
columns_subset = []
for t in ticker:
    columns_subset.append(f'{t}_target')


# Calculate the mean for the subset of columns
df['portfolio_target'] = df[columns_subset].mean(axis=1)

### IMPORTANT ###
# note that we can only minimise so we multiply the mean portfolio reaturn by -1
df['portfolio_target'] = df['portfolio_target'] * (-1)

df

,Date,Mkt-RF,SMB,HML,RF,AAPL,ABBV,AMZN,DB,DIS,...,GOOG_target,HAL_target,HSBC_target,JPM_target,KO_target,MCD_target,MSFT_target,PFE_target,XOM_target,portfolio_target
0,2015-01-05,-1.84,0.34,-0.68,0.0,-2.817130,-1.881942,-2.051729,-5.415164,-1.461356,...,-2.084562,-1.975207,-2.294430,-3.104486,0.000000,-1.104470,-0.919569,-0.542577,-2.736237,2.027776
1,2015-01-06,-1.04,-0.78,-0.30,0.0,0.009416,-0.494983,-2.283333,-1.596118,-0.530411,...,-2.317709,-0.981684,-1.456844,-2.592927,0.759365,0.184339,-1.467725,0.834400,-0.531599,0.890415
2,2015-01-07,1.19,0.18,-0.64,0.0,1.402210,4.041668,1.059974,1.057833,1.022978,...,-0.171323,2.687214,0.772271,0.152573,1.248225,1.742432,1.270507,1.368553,1.013224,-1.333453
3,2015-01-08,1.81,-0.12,-0.28,0.0,3.842247,1.045858,0.683602,0.209339,1.034136,...,0.315304,2.159561,0.328459,2.234649,1.209552,0.372315,2.941830,2.040809,1.664544,-1.434443
4,2015-01-09,-0.85,0.01,-0.48,0.0,0.107233,-2.735468,-1.174861,-1.671300,0.490457,...,-1.295055,-1.019654,-0.523771,-1.738685,-1.103178,-1.218759,-0.840491,0.461530,-0.141012,0.885929
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1355,2020-05-22,0.27,0.47,-0.87,0.0,0.643836,-0.989031,-0.402982,-0.543476,0.161245,...,0.543205,-3.255411,-5.860658,-0.776297,-0.309955,-0.361995,0.043611,0.644120,0.089767,0.741002
1356,2020-05-26,1.23,0.04,4.59,0.0,-0.677332,-1.509250,-0.616361,12.158465,2.482630,...,0.467937,2.415865,3.656941,7.097345,2.353986,0.233167,-1.057146,-0.026647,2.937226,-2.136916
1357,2020-05-27,1.54,0.61,3.62,0.0,0.435693,-0.804750,-0.473604,5.724739,0.479539,...,0.057868,2.695875,1.511959,5.792112,1.388598,1.558102,0.132166,-0.213403,0.718785,-1.357406
1358,2020-05-28,-0.41,-1.47,-2.41,0.0,0.043990,0.055573,-0.385415,-1.958531,-3.933184,...,-0.078283,-0.656276,-2.399655,-1.489591,0.770364,0.538034,-0.225520,2.058258,-2.595153,0.732528


In [22]:
model1 = LinearRegression()

# Define the independent variables (factors)
X1 = df[['Mkt-RF', 'SMB', 'HML']]  # Replace with the actual column names of your factors

# Define the dependent variable
y1 = df['portfolio_target']  # Replace with the actual column name of your dependent variable

# Fit the linear regression model
model1.fit(X1, y1)

predicted_values = model1.predict(X1)

df['FFF'] = predicted_values


df

,Date,Mkt-RF,SMB,HML,RF,AAPL,ABBV,AMZN,DB,DIS,...,HAL_target,HSBC_target,JPM_target,KO_target,MCD_target,MSFT_target,PFE_target,XOM_target,portfolio_target,FFF
0,2015-01-05,-1.84,0.34,-0.68,0.0,-2.817130,-1.881942,-2.051729,-5.415164,-1.461356,...,-1.975207,-2.294430,-3.104486,0.000000,-1.104470,-0.919569,-0.542577,-2.736237,2.027776,1.951976
1,2015-01-06,-1.04,-0.78,-0.30,0.0,0.009416,-0.494983,-2.283333,-1.596118,-0.530411,...,-0.981684,-1.456844,-2.592927,0.759365,0.184339,-1.467725,0.834400,-0.531599,0.890415,0.991979
2,2015-01-07,1.19,0.18,-0.64,0.0,1.402210,4.041668,1.059974,1.057833,1.022978,...,2.687214,0.772271,0.152573,1.248225,1.742432,1.270507,1.368553,1.013224,-1.333453,-1.092371
3,2015-01-08,1.81,-0.12,-0.28,0.0,3.842247,1.045858,0.683602,0.209339,1.034136,...,2.159561,0.328459,2.234649,1.209552,0.372315,2.941830,2.040809,1.664544,-1.434443,-1.787701
4,2015-01-09,-0.85,0.01,-0.48,0.0,0.107233,-2.735468,-1.174861,-1.671300,0.490457,...,-1.019654,-0.523771,-1.738685,-1.103178,-1.218759,-0.840491,0.461530,-0.141012,0.885929,0.905170
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1355,2020-05-22,0.27,0.47,-0.87,0.0,0.643836,-0.989031,-0.402982,-0.543476,0.161245,...,-3.255411,-5.860658,-0.776297,-0.309955,-0.361995,0.043611,0.644120,0.089767,0.741002,-0.115548
1356,2020-05-26,1.23,0.04,4.59,0.0,-0.677332,-1.509250,-0.616361,12.158465,2.482630,...,2.415865,3.656941,7.097345,2.353986,0.233167,-1.057146,-0.026647,2.937226,-2.136916,-1.822884
1357,2020-05-27,1.54,0.61,3.62,0.0,0.435693,-0.804750,-0.473604,5.724739,0.479539,...,2.695875,1.511959,5.792112,1.388598,1.558102,0.132166,-0.213403,0.718785,-1.357406,-1.949362
1358,2020-05-28,-0.41,-1.47,-2.41,0.0,0.043990,0.055573,-0.385415,-1.958531,-3.933184,...,-0.656276,-2.399655,-1.489591,0.770364,0.538034,-0.225520,2.058258,-2.595153,0.732528,0.566943


In [23]:

# Select 9 stocks from the dataset
selected_stocks = ['AAPL', 'ABBV', 'AMZN', 'DB', 'DIS', 'GOOG', 'HAL', 'HSBC', 'JPM']

# Extract the relevant columns for selected stocks from the DataFrame 'df'
selected_data = df[selected_stocks]

# Compute the covariance matrix using the selected stocks' returns
covariance_matrix = selected_data.cov()

# Define the objective function for portfolio optimization
def objective(weights, covariance_matrix):
    portfolio_variance = np.dot(weights.T, np.dot(covariance_matrix, weights))
    return portfolio_variance

# Define the constraint for portfolio weights summing to 1
def constraint(weights):
    return np.sum(weights) - 1.0

# Define the initial guess for portfolio weights
initial_weights = np.ones(len(selected_stocks)) / len(selected_stocks)

# Define the bounds for portfolio weights (0 <= weight <= 1)
bounds = [(0, 1)] * len(selected_stocks)

# Define the equality constraint for portfolio weights summing to 1
constraint_eq = {'type': 'eq', 'fun': constraint}

# Run the portfolio optimization to find the minimum variance portfolio
result = minimize(objective, initial_weights, args=(covariance_matrix,), method='SLSQP', bounds=bounds, constraints=constraint_eq)

# Get the optimal portfolio weights
optimal_weights = result.x

# Print the optimal weights for each stock
for stock, weight in zip(selected_stocks, optimal_weights):
    print(f"{stock}: {weight}")

# Calculate the portfolio return
portfolio_return = np.sum(selected_data * optimal_weights, axis=1)

# Print the portfolio return
print("Portfolio Return:", portfolio_return)


AAPL: 0.01608715596793819
ABBV: 0.15848988469833603
AMZN: 0.10361573463951752
DB: 1.6574232492103796e-16
DIS: 0.21868795863749152
GOOG: 0.06470719815127943
HAL: 0.0
HSBC: 0.4384120679054375
JPM: 0.0
Portfolio Return: 0      -2.016552
1      -1.219552
2       1.324151
3       0.688956
4      -0.759723
          ...   
1355   -2.687121
1356    1.862485
1357    0.601867
1358   -1.947663
1359   -0.203595
Length: 1360, dtype: float64


In [24]:
dataset_tr.rename(columns = {'Date_pnlog':'Date'}, inplace = True)

In [25]:
print(type(dataset_tr['Date'][0]))
print(type(inverted_df['Date'][0]))

<class 'str'>
<class 'pandas._libs.tslibs.timestamps.Timestamp'>


In [26]:
dataset_tr['Date'] = pd.to_datetime(dataset_tr['Date'], errors='coerce')
dataset_tr

,Date,AAPL_pnlog,ABBV_pnlog,AMZN_pnlog,DB_pnlog,DIS_pnlog,GOOG_pnlog,HAL_pnlog,HSBC_pnlog,JPM_pnlog,KO_pnlog,MCD_pnlog,MSFT_pnlog,PFE_pnlog,XOM_pnlog
0,2015-01-02,0.010336,-0.015882,-0.020067,0.004023,0.008930,0.007672,0.001855,-0.037944,0.022862,NaN,-0.001835,0.011189,-0.015655,NaN
1,2015-01-05,0.014229,0.000000,0.013374,0.008442,0.010258,-0.002559,-0.012387,-0.023388,0.007260,0.000000,-0.001565,0.013213,0.000000,-0.023893
2,2015-01-06,0.017950,0.010092,0.019457,-0.005736,-0.016278,0.017585,0.019216,-0.035777,-0.023960,0.000000,-0.037644,0.017114,0.010571,-0.026224
3,2015-01-07,0.000000,0.010801,-0.010497,0.010894,-0.002005,-0.004112,0.003567,-0.021697,-0.013968,0.000000,-0.017531,0.020706,0.012930,0.000000
4,2015-01-08,0.005537,0.007774,0.010360,-0.003153,0.041047,0.015942,0.001991,-0.009976,0.001864,0.007091,-0.000143,0.028966,0.008093,-0.003405
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1363,2020-06-03,0.000284,-0.027009,-0.017542,0.000000,-0.039138,-0.030677,-0.016068,-0.017556,0.006868,0.000000,0.000000,0.000000,-0.002122,NaN
1364,2020-06-04,-0.004683,-0.033180,-0.014703,0.000000,-0.007554,-0.023413,-0.016782,-0.012212,-0.006033,NaN,0.000000,-0.020826,-0.016932,-0.011122
1365,2020-06-05,-0.009095,0.059226,-0.031269,-0.013804,-0.007840,-0.011109,-0.009981,-0.015163,-0.004168,0.004322,0.004753,-0.016721,0.000997,-0.017159
1366,2020-06-08,0.001563,NaN,0.007215,0.035353,0.000000,0.001961,-0.017587,-0.024399,-0.007609,0.000000,0.000000,-0.004798,0.009710,0.000000


In [27]:
df_q2 = pd.merge(df, dataset_tr, how='outer', on='Date')
df_q2

,Date,Mkt-RF,SMB,HML,RF,AAPL,ABBV,AMZN,DB,DIS,...,DIS_pnlog,GOOG_pnlog,HAL_pnlog,HSBC_pnlog,JPM_pnlog,KO_pnlog,MCD_pnlog,MSFT_pnlog,PFE_pnlog,XOM_pnlog
0,2015-01-05,-1.84,0.34,-0.68,0.0,-2.817130,-1.881942,-2.051729,-5.415164,-1.461356,...,0.010258,-0.002559,-0.012387,-0.023388,0.007260,0.000000,-0.001565,0.013213,0.000000,-0.023893
1,2015-01-06,-1.04,-0.78,-0.30,0.0,0.009416,-0.494983,-2.283333,-1.596118,-0.530411,...,-0.016278,0.017585,0.019216,-0.035777,-0.023960,0.000000,-0.037644,0.017114,0.010571,-0.026224
2,2015-01-07,1.19,0.18,-0.64,0.0,1.402210,4.041668,1.059974,1.057833,1.022978,...,-0.002005,-0.004112,0.003567,-0.021697,-0.013968,0.000000,-0.017531,0.020706,0.012930,0.000000
3,2015-01-08,1.81,-0.12,-0.28,0.0,3.842247,1.045858,0.683602,0.209339,1.034136,...,0.041047,0.015942,0.001991,-0.009976,0.001864,0.007091,-0.000143,0.028966,0.008093,-0.003405
4,2015-01-09,-0.85,0.01,-0.48,0.0,0.107233,-2.735468,-1.174861,-1.671300,0.490457,...,0.005140,-0.006977,0.000000,-0.035810,0.000000,0.000000,0.000000,0.005702,-0.018554,-0.089498
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1363,2020-06-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.039138,-0.030677,-0.016068,-0.017556,0.006868,0.000000,0.000000,0.000000,-0.002122,NaN
1364,2020-06-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.007554,-0.023413,-0.016782,-0.012212,-0.006033,NaN,0.000000,-0.020826,-0.016932,-0.011122
1365,2020-06-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.007840,-0.011109,-0.009981,-0.015163,-0.004168,0.004322,0.004753,-0.016721,0.000997,-0.017159
1366,2020-06-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.000000,0.001961,-0.017587,-0.024399,-0.007609,0.000000,0.000000,-0.004798,0.009710,0.000000


In [28]:
df_q2 = df_q2.dropna()

In [29]:
newcolumns_subset = dataset_tr.columns
newcolumns_subset = newcolumns_subset.to_list()
newcolumns_subset = newcolumns_subset[1:]
newcolumns_subset
df_q2['avg_sentiment'] = df_q2[newcolumns_subset].mean(axis=1)

/var/folders/hj/n94x31sj08bdmgwjs6ybxbf40000gn/T/ipykernel_7600/534143878.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_q2['avg_sentiment'] = df_q2[newcolumns_subset].mean(axis=1)


In [30]:
df_q2

,Date,Mkt-RF,SMB,HML,RF,AAPL,ABBV,AMZN,DB,DIS,...,GOOG_pnlog,HAL_pnlog,HSBC_pnlog,JPM_pnlog,KO_pnlog,MCD_pnlog,MSFT_pnlog,PFE_pnlog,XOM_pnlog,avg_sentiment
0,2015-01-05,-1.84,0.34,-0.68,0.0,-2.817130,-1.881942,-2.051729,-5.415164,-1.461356,...,-0.002559,-0.012387,-0.023388,0.007260,0.000000,-0.001565,0.013213,0.000000,-0.023893,0.000213
1,2015-01-06,-1.04,-0.78,-0.30,0.0,0.009416,-0.494983,-2.283333,-1.596118,-0.530411,...,0.017585,0.019216,-0.035777,-0.023960,0.000000,-0.037644,0.017114,0.010571,-0.026224,-0.002402
2,2015-01-07,1.19,0.18,-0.64,0.0,1.402210,4.041668,1.059974,1.057833,1.022978,...,-0.004112,0.003567,-0.021697,-0.013968,0.000000,-0.017531,0.020706,0.012930,0.000000,-0.000779
3,2015-01-08,1.81,-0.12,-0.28,0.0,3.842247,1.045858,0.683602,0.209339,1.034136,...,0.015942,0.001991,-0.009976,0.001864,0.007091,-0.000143,0.028966,0.008093,-0.003405,0.007999
4,2015-01-09,-0.85,0.01,-0.48,0.0,0.107233,-2.735468,-1.174861,-1.671300,0.490457,...,-0.006977,0.000000,-0.035810,0.000000,0.000000,0.000000,0.005702,-0.018554,-0.089498,-0.010345
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1353,2020-05-20,1.80,1.07,1.29,0.0,1.944802,-0.010968,1.984628,6.373943,4.852667,...,0.000713,-0.023388,-0.042742,-0.012811,0.000000,-0.020342,0.003391,-0.002848,-0.037823,-0.010527
1354,2020-05-21,-0.70,0.65,0.41,0.0,-0.745538,2.006799,-2.049693,-1.997341,-1.742826,...,0.001555,-0.010249,-0.035370,-0.003717,0.000000,0.000000,-0.004664,0.017583,0.000000,-0.012118
1355,2020-05-22,0.27,0.47,-0.87,0.0,0.643836,-0.989031,-0.402982,-0.543476,0.161245,...,0.014212,-0.080930,-0.034325,-0.005740,0.000000,-0.007848,0.001411,0.009057,-0.017679,-0.017079
1358,2020-05-28,-0.41,-1.47,-2.41,0.0,0.043990,0.055573,-0.385415,-1.958531,-3.933184,...,-0.027624,-0.016211,-0.012882,-0.012568,0.000000,-0.048097,-0.013848,0.000711,-0.016618,-0.019674


In [31]:
model2 = LinearRegression()

X2 = df_q2[['Mkt-RF', 'SMB', 'HML', 'avg_sentiment']]  

# Define the dependent variable
y2 = df_q2['portfolio_target']  

# Fit the linear regression model
model2.fit(X2, y2)

predicted_values2 = model2.predict(X2)

df_q2['FFFF'] = predicted_values2


df_q2

/var/folders/hj/n94x31sj08bdmgwjs6ybxbf40000gn/T/ipykernel_7600/990700993.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_q2['FFFF'] = predicted_values2


,Date,Mkt-RF,SMB,HML,RF,AAPL,ABBV,AMZN,DB,DIS,...,HAL_pnlog,HSBC_pnlog,JPM_pnlog,KO_pnlog,MCD_pnlog,MSFT_pnlog,PFE_pnlog,XOM_pnlog,avg_sentiment,FFFF
0,2015-01-05,-1.84,0.34,-0.68,0.0,-2.817130,-1.881942,-2.051729,-5.415164,-1.461356,...,-0.012387,-0.023388,0.007260,0.000000,-0.001565,0.013213,0.000000,-0.023893,0.000213,1.882652
1,2015-01-06,-1.04,-0.78,-0.30,0.0,0.009416,-0.494983,-2.283333,-1.596118,-0.530411,...,0.019216,-0.035777,-0.023960,0.000000,-0.037644,0.017114,0.010571,-0.026224,-0.002402,0.933705
2,2015-01-07,1.19,0.18,-0.64,0.0,1.402210,4.041668,1.059974,1.057833,1.022978,...,0.003567,-0.021697,-0.013968,0.000000,-0.017531,0.020706,0.012930,0.000000,-0.000779,-1.068746
3,2015-01-08,1.81,-0.12,-0.28,0.0,3.842247,1.045858,0.683602,0.209339,1.034136,...,0.001991,-0.009976,0.001864,0.007091,-0.000143,0.028966,0.008093,-0.003405,0.007999,-1.809191
4,2015-01-09,-0.85,0.01,-0.48,0.0,0.107233,-2.735468,-1.174861,-1.671300,0.490457,...,0.000000,-0.035810,0.000000,0.000000,0.000000,0.005702,-0.018554,-0.089498,-0.010345,0.897017
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1353,2020-05-20,1.80,1.07,1.29,0.0,1.944802,-0.010968,1.984628,6.373943,4.852667,...,-0.023388,-0.042742,-0.012811,0.000000,-0.020342,0.003391,-0.002848,-0.037823,-0.010527,-1.927967
1354,2020-05-21,-0.70,0.65,0.41,0.0,-0.745538,2.006799,-2.049693,-1.997341,-1.742826,...,-0.010249,-0.035370,-0.003717,0.000000,0.000000,-0.004664,0.017583,0.000000,-0.012118,0.638310
1355,2020-05-22,0.27,0.47,-0.87,0.0,0.643836,-0.989031,-0.402982,-0.543476,0.161245,...,-0.080930,-0.034325,-0.005740,0.000000,-0.007848,0.001411,0.009057,-0.017679,-0.017079,-0.033445
1358,2020-05-28,-0.41,-1.47,-2.41,0.0,0.043990,0.055573,-0.385415,-1.958531,-3.933184,...,-0.016211,-0.012882,-0.012568,0.000000,-0.048097,-0.013848,0.000711,-0.016618,-0.019674,0.765070


In [34]:
num_perturbations = 10

kappa_values = [0.2, 0.5, 0.8]

# Perform robust portfolio optimization with different perturbation matrices and kappa values
for kappa in kappa_values:
    print(f"Kappa: {kappa}")
    
    for i in range(num_perturbations):
        print(f"Perturbation {i + 1}/{num_perturbations}")
        
        # Generate a perturbation matrix (Sigma) using the factor model returns and kappa
        perturbation_matrix = np.diag(kappa * np.abs(df_q2['FFFF']))#[:, 0]))  # Adjust the perturbation matrix based on your factor model
        
        # Define the objective function for portfolio optimization
        def objective(weights):
            portfolio_return = np.dot(weights.T, df_q2['FFFF'])
            return -portfolio_return

        # Define the constraint for portfolio weights summing to 1
        def constraint(weights):
            return np.sum(weights) - 1.0

        # Define the initial guess for portfolio weights
        initial_weights = np.ones(len(selected_stocks)) / len(selected_stocks)

        # Define the bounds for portfolio weights (0 <= weight <= 1)
        bounds = [(0, 1)] * len(selected_stocks)

        # Define the equality constraint for portfolio weights summing to 1
        constraint_eq = {'type': 'eq', 'fun': constraint}

        # Run the portfolio optimization to find the maximum return portfolio
        result = minimize(objective, initial_weights, method='SLSQP', bounds=bounds, constraints=constraint_eq)

        # Get the optimal portfolio weights
        optimal_weights = result.x

        # Print the optimal weights for each stock
        for stock, weight in zip(selected_stocks, optimal_weights):
            print(f"{stock}: {weight}")
        
        # Calculate the portfolio return
        portfolio_return = np.dot(optimal_weights.T, df_q2['FFFF'])[0, 0]
        
        # Print the portfolio return
        print("Portfolio Return:", portfolio_return)
        print()


Kappa: 0.2
Perturbation 1/10


ValueError: shapes (9,) and (327,) not aligned: 9 (dim 0) != 327 (dim 0)

In [33]:
num_perturbations = 10
kappa_values = [0.2, 0.5, 0.8]

# Define your selected stocks
selected_stocks = ['AAPL', 'ABBV', 'AMZN', 'DB', 'DIS', 'GOOG', 'HAL', 'HSBC', 'JPM']

# Perform robust portfolio optimization with different perturbation matrices and kappa values
for kappa in kappa_values:
    print(f"Kappa: {kappa}")
    
    for i in range(num_perturbations):
        print(f"Perturbation {i + 1}/{num_perturbations}")
        
        # Generate a perturbation matrix (Sigma) using the factor model returns and kappa
        perturbation_matrix = np.diag(kappa * np.abs(df_q2['FFFF']))
        
        # Define the objective function for portfolio optimization
        def objective(weights):
            # Calculate the portfolio return for each date using the weights and selected stocks
            portfolio_return_series = df_q2[selected_stocks].multiply(weights).sum(axis=1)
            # Multiply the portfolio returns by the FFF model predictions
            portfolio_return = np.dot(portfolio_return_series, df_q2['FFFF'])
            # We want to maximize the return, so we return the negative
            return -portfolio_return

        # Define the constraint for portfolio weights summing to 1
        def constraint(weights):
            return np.sum(weights) - 1.0

        # Define the initial guess for portfolio weights
        initial_weights = np.ones(len(selected_stocks)) / len(selected_stocks)

        # Define the bounds for portfolio weights (0 <= weight <= 1)
        bounds = [(0, 1)] * len(selected_stocks)

        # Define the equality constraint for portfolio weights summing to 1
        constraint_eq = {'type': 'eq', 'fun': constraint}

        # Run the portfolio optimization to find the maximum return portfolio
        result = minimize(objective, initial_weights, method='SLSQP', bounds=bounds, constraints=constraint_eq)

        # Get the optimal portfolio weights
        optimal_weights = result.x

        # Print the optimal weights for each stock
        for stock, weight in zip(selected_stocks, optimal_weights):
            print(f"{stock}: {weight}")
        
        # Calculate the portfolio return
        portfolio_return_series = df_q2[selected_stocks].multiply(optimal_weights).sum(axis=1)
        portfolio_return = np.dot(portfolio_return_series, df_q2['FFFF'])
        
        # Print the portfolio return
        print("Portfolio Return:", portfolio_return)
        print()

Kappa: 0.2
Perturbation 1/10
AAPL: 0.0
ABBV: 4.869171175826537e-09
AMZN: 1.0
DB: 0.0
DIS: 1.682614351862357e-09
GOOG: 2.1413833746332216e-09
HAL: 0.0
HSBC: 4.318244746349798e-09
JPM: 0.0
Portfolio Return: -628.6382395016911

Perturbation 2/10
AAPL: 0.0
ABBV: 4.869171175826537e-09
AMZN: 1.0
DB: 0.0
DIS: 1.682614351862357e-09
GOOG: 2.1413833746332216e-09
HAL: 0.0
HSBC: 4.318244746349798e-09
JPM: 0.0
Portfolio Return: -628.6382395016911

Perturbation 3/10
AAPL: 0.0
ABBV: 4.869171175826537e-09
AMZN: 1.0
DB: 0.0
DIS: 1.682614351862357e-09
GOOG: 2.1413833746332216e-09
HAL: 0.0
HSBC: 4.318244746349798e-09
JPM: 0.0
Portfolio Return: -628.6382395016911

Perturbation 4/10
AAPL: 0.0
ABBV: 4.869171175826537e-09
AMZN: 1.0
DB: 0.0
DIS: 1.682614351862357e-09
GOOG: 2.1413833746332216e-09
HAL: 0.0
HSBC: 4.318244746349798e-09
JPM: 0.0
Portfolio Return: -628.6382395016911

Perturbation 5/10
AAPL: 0.0
ABBV: 4.869171175826537e-09
AMZN: 1.0
DB: 0.0
DIS: 1.682614351862357e-09
GOOG: 2.1413833746332216e-09
HAL